In [1]:
import sys
import time

from geopy.geocoders import Nominatim

import pyspark.sql.functions as f
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import lower, col, size, length
from operator import add
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer

import pandas as pd
import numpy as np

spark = SparkSession\
  .builder \
  .appName("Twitter_app") \
  .getOrCreate()

### Loading Stop Words:

In [2]:
## Reading from the stopword file:
text_file = open("datasets/stop_words_english.txt", "r")
lines = text_file.read()

## Creating the stop word array:
stopWords = lines.split()
# Adding http and https to it:
stopWords.append('http')
stopWords.append('https')
stopWords[-10:]

['whim',
 'whod',
 'whos',
 'widely',
 'words',
 'world',
 'youd',
 'youre',
 'http',
 'https']

### Loading the Data:

In [3]:
#Load the data
data = spark.read.format('json').options(header='true', inferSchema='true') \
  .load('./datasets/NoFilterEnglish2020-02-04.json')

### Exercise 1

##### Words preprocessed:

In [36]:
# Clean text
clean_texts = data.select('*', lower(f.regexp_replace(f.col('text'), r'[^a-zA-Z#@,!\\s]', ' ')).alias('text2'))\
                  .drop('text')\
                  .withColumnRenamed('text2', 'text')

clean_texts.select('text').show(1, False)

+---------------------------------------------------------+
|text                                                     |
+---------------------------------------------------------+
|@theythemsbian thank you for being brave enough to say it|
+---------------------------------------------------------+
only showing top 1 row



#### Tokenize the words:

In [37]:
# Tokenize text
tokenizer = Tokenizer(inputCol='text', outputCol='words_t')
clean_word_tokens = tokenizer.transform(clean_texts)

clean_word_tokens.select('words_t').show(1, False)

+--------------------------------------------------------------------+
|words_t                                                             |
+--------------------------------------------------------------------+
|[@theythemsbian, thank, you, for, being, brave, enough, to, say, it]|
+--------------------------------------------------------------------+
only showing top 1 row



#### Remove useless words:

By useless, we mean the words of size 2 or less.

In [38]:
## Defining the function: (udf: user defined function)
filter_length_udf = f.udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))
clean_word_tokens = clean_word_tokens.withColumn('words', filter_length_udf(col('words_t')))

clean_word_tokens.select('words').show(1, False)

+------------------------------------------------------------+
|words                                                       |
+------------------------------------------------------------+
|[@theythemsbian, thank, you, for, being, brave, enough, say]|
+------------------------------------------------------------+
only showing top 1 row



#### Creating the view words:

In [39]:
exp_words = clean_word_tokens.withColumn('word', f.explode('words'))\
                             .drop('words')
                 
#Tokenizer
exp_words.select('text', 'word').show(4)

exp_words.createOrReplaceTempView("words")



+--------------------+--------------+
|                text|          word|
+--------------------+--------------+
|@theythemsbian th...|@theythemsbian|
|@theythemsbian th...|         thank|
|@theythemsbian th...|           you|
|@theythemsbian th...|           for|
+--------------------+--------------+
only showing top 4 rows



#### Getting most frequent words in a given period of time using SPARK SQL:

In [40]:
def FreqWords(ts1, ts2, table = "words"):

    sqlDF = spark.sql("SELECT word, count(*) AS word_count " + 
                      "FROM {} ".format(table) + 
                      "WHERE {0}.timestamp_ms BETWEEN {1} AND {2} ".format(table, ts1, ts2)+ 
                      "GROUP BY word " + 
                      "ORDER BY word_count DESC")
    sqlDF.show(10)
    return sqlDF

In [41]:
ts1 = 1580770795658
ts2 = 1580770796665


t1 = time.time()
FreqWords(ts1, ts2, table = "words")
t2 = time.time()

print("time spent computing: {:.4g}".format(t2-t1))

+-----+----------+
| word|word_count|
+-----+----------+
|  the|        13|
|https|         7|
|  you|         6|
| this|         6|
|  for|         4|
|  can|         4|
| make|         3|
| need|         3|
|  not|         2|
| rare|         2|
+-----+----------+
only showing top 10 rows

time spent computing: 5.339


#### Getting most frequent words in a given period of time using SPARK only:

In [42]:
ts1 = 1580770795658
ts2 = 1580770796665

## Timing the operation:
t1 = time.time()

exp_words.filter(f.col('timestamp_ms').between(ts1, ts2) )\
         .groupBy('word') \
         .count() \
         .sort('count', ascending=False) \
         .show(10)

t2 = time.time()

#         .filter(length(col("word")) >= 3)\

print("time spent computing: {:.4g}".format(t2-t1))

+-----+-----+
| word|count|
+-----+-----+
|  the|   13|
|https|    7|
|  you|    6|
| this|    6|
|  for|    4|
|  can|    4|
| make|    3|
| need|    3|
| rare|    2|
|  not|    2|
+-----+-----+
only showing top 10 rows

time spent computing: 5.297


##### Conclusion:

We can see that both methods are equivalent in computing time and yield to the same results. Choosing between both is just a question of taste. We Personally prefer **SQL** querying.

### Exercise 2

#### Removing stop words:

In [43]:
# Removing the stopwords from the array of strings
sc = spark.sparkContext
broadcastVar = sc.broadcast(stopWords)
broadcastVar.value

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=broadcastVar.value)
words_f = remover.transform(clean_word_tokens)
words_f = words_f.drop('words_t', 'words')

words_f.select('filtered').show(5, False)

+-----------------------------------------------------------------------------------------------------------+
|filtered                                                                                                   |
+-----------------------------------------------------------------------------------------------------------+
|[@theythemsbian, brave]                                                                                    |
|[@camillediola, duterte, violated, law, appointing, honasan,, background,, head, dict, shrugged,, pointing]|
|[@tinyseokjinnie, lol, bored, dypgt]                                                                       |
|[@ibesuckafree, suck, quitting, weed, argument, amp, gettin]                                               |
|[@mikebloomberg, donald, trump, bring, change, country, #superbowl, vciycilow]                             |
+-----------------------------------------------------------------------------------------------------------+
only showi

In [44]:
words_f.columns[-5:]

['truncated', 'user', 'withheld_in_countries', 'text', 'filtered']

#### Creating the view words_filtered:

In [45]:
exp_words_f = words_f.withColumn('word', f.explode('filtered'))#\
                     #.drop('filtered')
#.drop('filtered')???

exp_words_f.select('text', 'word').show(1, False)

exp_words_f.createOrReplaceTempView("words_filtered")

+---------------------------------------------------------+--------------+
|text                                                     |word          |
+---------------------------------------------------------+--------------+
|@theythemsbian thank you for being brave enough to say it|@theythemsbian|
+---------------------------------------------------------+--------------+
only showing top 1 row



In [46]:
exp_words_f.columns[-5:]

['user', 'withheld_in_countries', 'text', 'filtered', 'word']

#### Getting most frequent words in a given period of time after filtering using SPARK SQL:

In [47]:
ts1 = 1580770795658
ts2 = 1580770796665


t1 = time.time()
FreqWords(ts1, ts2, table = "words_filtered")
t2 = time.time()

print("time spent computing: {:.4g}".format(t2-t1))

+-------------+----------+
|         word|word_count|
+-------------+----------+
|          pro|         2|
|         rare|         2|
|         rush|         2|
|          amp|         2|
|         fuck|         2|
|    @yungcalc|         1|
|    @santeira|         1|
|        paral|         1|
|@chqmbiedolan|         1|
|     whaaaaat|         1|
+-------------+----------+
only showing top 10 rows

time spent computing: 5.306


### Exercise 3

In [17]:
exp_words_f.filter("(timestamp_ms / 1000 / 60 / 60  % 24 )>= 20") \
           .groupBy('word') \
           .count() \
           .sort('count', ascending=False) \
           .show()

+----------------+-----+
|            word|count|
+----------------+-----+
|             don| 7919|
|             amp| 7470|
|          people| 6621|
|            love| 6304|
|            time| 5231|
|             day| 4775|
|           trump| 4621|
|            good| 4434|
|           today| 3708|
|            iowa| 3488|
|            shit| 3213|
|             man| 3156|
|            life| 2527|
|           happy| 2424|
|            fuck| 2337|
|            year| 2273|
|           years| 2206|
|           black| 2127|
|@realdonaldtrump| 2107|
|           great| 2065|
+----------------+-----+
only showing top 20 rows



### Exercise 5

In [18]:
hashtags = exp_words_f.filter( exp_words_f.word.startswith('#'))\
                      .withColumnRenamed('word', 'hashtag')

In [19]:
most_freq_hash = hashtags.groupBy('hashtag') \
    .count() \
    .sort('count', ascending=False) 
most_freq_hash.show(30)

+--------------------+-----+
|             hashtag|count|
+--------------------+-----+
|         #curecancer| 3987|
|       #iowacaucuses| 3253|
|#deshkiawaazshehnaaz| 2871|
|    #mainbhiasimkigf| 2782|
|      #meraadarshsid| 2685|
|       #iheartawards| 1753|
|                #bts| 1658|
|           #asimriaz| 1512|
|        #thebachelor| 1358|
|         #iowacaucus| 1274|
|                 #bb| 1251|
|     #sidharthshukla| 1245|
|           #treasure| 1232|
|        #coronavirus| 1197|
|           #iacaucus| 1197|
|           #biggboss| 1156|
|         #loveisland|  999|
|          #superbowl|  976|
|     #worldcancerday|  965|
|               #sotu|  894|
|     #rightchoicesid|  843|
|              #yikes|  776|
|     #happyjisungday|  710|
|               #iowa|  696|
|            #btsarmy|  686|
|         #nowplaying|  646|
|                #nct|  640|
|       #rashamidesai|  582|
|                #raw|  581|
|             #superm|  574|
+--------------------+-----+
only showing t

###### Converting Hashtag Data Frame to array:

In [20]:
mfht = most_freq_hash.filter('count >= 500').select('hashtag').rdd.map(lambda x: x[0]).collect()

###### Getting words related to Hashtag example:

In [21]:
event_words = exp_words_f.filter( f.col('text').contains('#iowacaucuses'))

event_words.groupBy('word') \
           .count() \
           .sort('count', ascending=False)\
           .rdd\
           .map(lambda x: x[0])\
           .collect()[:12]

['#iowacaucuses',
 'iowa',
 'bernie',
 'caucus',
 'vote',
 'won',
 'tonight',
 '#iowacaucus',
 'biden',
 'app',
 '@donaldjtrumpjr',
 'dnc']

#### Ex5 using pySpark:

In [87]:
## Ex.5: Get the most frequent words for each event:
ht_f = hashtags.select('filtered', 'hashtag')\
                         .filter(f.col('hashtag').isin(mfht))\
                         .withColumn('word', f.explode('filtered'))\
                         .filter(~col('word').startswith('#'))\
                         .select('hashtag', 'word')\
                         .groupBy('hashtag', 'word')\
                         .count()\
                         .sort(['hashtag', 'count'], ascending=False)\
                         .filter('count >= 100')

ht_f.show(10)
    

+-------+-----------+-----+
|hashtag|       word|count|
+-------+-----------+-----+
| #yikes|@nickiminaj|  465|
| #yikes|      likes|  348|
| #yikes|        amp|  336|
| #yikes|       play|  325|
| #yikes|       life|  313|
| #yikes|        tag|  313|
| #yikes|    yzdifvf|  310|
| #yikes|       rosa|  137|
| #yikes|      parks|  129|
| #yikes|      nicki|  121|
+-------+-----------+-----+
only showing top 10 rows



#### Showing the top 10 most frequent words by events:

In [ ]:
for ht in mfht:
    ht_f.filter(col('hashtag').isin(ht)).show(10)

+-----------+-----------+-----+
|    hashtag|       word|count|
+-----------+-----------+-----+
|#curecancer|trueworship| 3951|
|#curecancer|     cancer| 2441|
|#curecancer|       true| 1240|
|#curecancer|     rampal| 1073|
|#curecancer|    worship| 1042|
|#curecancer|      saint| 1040|
|#curecancer|       cure|  916|
|#curecancer|        god|  884|
|#curecancer|    maharaj|  859|
|#curecancer|   diseases|  846|
+-----------+-----------+-----+
only showing top 10 rows

+-------------+---------------+-----+
|      hashtag|           word|count|
+-------------+---------------+-----+
|#iowacaucuses|           iowa|  499|
|#iowacaucuses|         bernie|  346|
|#iowacaucuses|         caucus|  237|
|#iowacaucuses|           vote|  193|
|#iowacaucuses|            won|  173|
|#iowacaucuses|        tonight|  155|
|#iowacaucuses|          biden|  153|
|#iowacaucuses|            app|  151|
|#iowacaucuses|@donaldjtrumpjr|  150|
|#iowacaucuses|            dnc|  147|
+-------------+---------------+-

### Function that converts geolocalisation to place name:

In [25]:
def loc(coords):

    geolocator = Nominatim(user_agent="geoapiExercises")  
    if coords.shape != (2,):
        coords = coords.mean(axis = 0)
    
    Longitude = str(coords[0])
    Latitude = str(coords[1])
    
    location = geolocator.reverse(Latitude+","+Longitude)
    
    if(location == None):
        print("No location specified")
        return
 
    address = location.raw['address']

    # traverse the data
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    code = address.get('country_code')
    zipcode = address.get('postcode')
    print('City : ', city)
    print('State : ', state)
    print('Country : ', country)
    print('Zip Code : ', zipcode)

### Exercise 6 and Exercice 7

In [27]:
for ht in mfht:
    event_words = exp_words_f.filter( f.col('text').contains(ht))
    event_texts = clean_texts.filter( f.col('text').contains(ht))

    ## Ex.5: Get the most frequent words for each event:
    most_freq_w = event_words.groupBy('word') \
                             .count() \
                             .sort('count', ascending=False)\
                             .rdd\
                             .map(lambda x: x[0])\
                             .collect()[:12]    
    
    # Ex.6 : Finding timeframe of the Event
    end   = event_texts.select(f.max(f.col('timestamp_ms'))).rdd.map(list).collect()[0][0]
    start = event_texts.select(f.min(f.col('timestamp_ms'))).rdd.map(list).collect()[0][0]
    
    # Ex.7 : Finding location of the Event
    coords = event_texts.na.drop(subset=["place"]) \
                        .select("place.bounding_box.coordinates") \
                        .rdd.map(list).collect()
    
    ## Gets the most common location from the tweets !
    location_name = event_texts_na.groupBy('place.name', 'place.country') \
                                  .count()\
                                  .sort('count', ascending=False)\
                                  .rdd\
                                  .map(lambda x : x[0] + ', ' + x[1])\
                                  .collect()
    

    if(coords != []):
        coords_mean = np.array(coords).squeeze().mean(axis = 0)
        loc(coords_mean)
    else: 
        print("No location specified")
    
    if(most_freq_w == []):
        most_freq_w = 'no words found...'
    
    if(location_name != []):
        location_name = location_name[0]
    else:
        location_name = 'No Location specified'
    

    end = str( pd.to_datetime(end, unit='ms').to_pydatetime())
    start = str(pd.to_datetime(start, unit='ms').to_pydatetime())
    print("Event : " + ht)
    print("Timeframe : from - " + start[:-7] + " to - " + end[:-7])
    print("Most frequent Location is : " + location_name)
    print("Most frequent words associated to the event: \n" + str(most_freq_w) + '\n\n')

City :  
State :  Madhya Pradesh
Country :  India
Zip Code :  476332
Event : #curecancer
Timeframe : from - 2020-02-04 03:16:13 to - 2020-02-04 22:42:39
Most frequent Location is : Mumbai, India
Most frequent words associated to the event: 
['#curecancer', 'trueworship', 'cancer', 'true', 'rampal', 'saint', 'worship', 'cure', 'god', 'maharaj', 'diseases', 'disease']


City :  
State :  West Virginia
Country :  United States
Zip Code :  None
Event : #iowacaucuses
Timeframe : from - 2020-02-03 23:00:11 to - 2020-02-04 22:58:28
Most frequent Location is : Mumbai, India
Most frequent words associated to the event: 
['#iowacaucuses', 'iowa', 'bernie', 'caucus', 'vote', 'won', 'tonight', '#iowacaucus', 'biden', 'app', '@donaldjtrumpjr', 'dnc']


City :  
State :  Maharashtra
Country :  India
Zip Code :  431201
Event : #deshkiawaazshehnaaz
Timeframe : from - 2020-02-04 01:58:49 to - 2020-02-04 22:59:12
Most frequent Location is : Mumbai, India
Most frequent words associated to the event: 
['#

In [25]:
# Method 1: Use describe()
t1 = time.time()
float(event_words.describe("timestamp_ms").filter("summary = 'max'").select("timestamp_ms").collect()[0].asDict()['timestamp_ms'])

t2 = time.time()
print("Method 1")
print("time spent computing: {:.4g}".format(t2-t1))

# Method 2: Use SQL
t1 = time.time()

event_words.registerTempTable("df_table")
spark.sql("SELECT MAX(timestamp_ms) as maxval FROM df_table").collect()[0].asDict()['maxval']

t2 = time.time()
print("Method 2")
print("time spent computing: {:.4g}".format(t2-t1))

# Method 3: Convert to RDD
t1 = time.time()

event_words.select("timestamp_ms").rdd.max()[0]

t2 = time.time()
print("Method 3")
print("time spent computing: {:.4g}".format(t2-t1))


# Method 4: select
t1 = time.time()

event_words.select(f.max(f.col('timestamp_ms')))

t2 = time.time()
print("Method 4")
print("time spent computing: {:.4g}".format(t2-t1))

Method 1
time spent computing: 6.778
Method 2
time spent computing: 6.634
Method 3
time spent computing: 6.558
Method 4
time spent computing: 0.009586


## Question 8:

Use NLP package from [nltk](https://www.nltk.org/api/nltk.sentiment.html).

In [54]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
#sid.polarity_scores(sentence)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /users/eleves-a/2018/jean-charles.layoun/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### Sentiment analysis most frequent hashtags:

Following the **map reduce** paradigm to compute the sentiment associated with each event.

###### Map by hashtags:

In [55]:
sentiments_rdd = hashtags.select('filtered', 'hashtag')\
                         .filter(f.col('hashtag').isin(mfht))\
                         .rdd.map(lambda x: (x[1], sid.polarity_scores(' '.join(x[0]))))

## Because we want to reduce and aggregate with respect to each event(hashtag), we choose hashtag to be our key:
# sentiments_rdd has (key=hashtag, value=sentiment)
t1 = time.time()
sentiments_rdd.collect()
t2 = time.time()
time1 = t2-t1

In [56]:
sentiments_rdd.take(10)

[('#superbowl', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#btsarmy', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#coronavirus', {'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compound': 0.5574}),
 ('#coronavirus', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#thebachelor', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#iowacaucuses', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#bts', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#nowplaying', {'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.2732}),
 ('#bts', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}),
 ('#giveaway', {'neg': 0.0, 'neu': 0.743, 'pos': 0.257, 'compound': 0.5859})]

###### Reduce on Hashtags:

In [57]:
def aggregate_sentiments(x, y):
    dict1 = x
    dict2 = y    
    
    for key in dict1.keys():
        dict1[key] += dict2[key]
        
    return dict1
    

aggreg = sentiments_rdd.reduceByKey(lambda x, y: aggregate_sentiments(x, y))

In [58]:

t1 = time.time()
event_sentiments_l = aggreg.collect()
t2 = time.time()
time2 = t2-t1

event_sentiments_l[:5]

[('#treasure',
  {'neg': 4.951999999999999,
   'neu': 1030.175000000003,
   'pos': 196.87100000000038,
   'compound': 349.67990000000117}),
 ('#arsd',
  {'neg': 35.58,
   'neu': 451.28099999999995,
   'pos': 50.13499999999999,
   'compound': 35.2231}),
 ('#iowa',
  {'neg': 73.184,
   'neu': 521.3799999999999,
   'pos': 101.45099999999998,
   'compound': 55.191799999999986}),
 ('#biggboss',
  {'neg': 61.392999999999994,
   'neu': 872.8369999999999,
   'pos': 221.77900000000002,
   'compound': 289.0983999999999}),
 ('#thebachelor',
  {'neg': 133.68300000000002,
   'neu': 1076.398,
   'pos': 147.925,
   'compound': 23.1815})]

In [59]:
def normalize_sentiment(x):
    dict1    = x
    sum_prob = dict1['neg'] + dict1['neu'] + dict1['pos']
    
    for key in dict1.keys():
        dict1[key] /= sum_prob
    return dict1


aggreg_normalized = aggreg.map(lambda x: (x[0], normalize_sentiment(x[1])))

In [60]:
t1 = time.time()
event_sentiments_ln = aggreg_normalized.collect()
t2 = time.time()
time3 = t2-t1

In [61]:
event_sentiments_ln[:8]

[('#treasure',
  {'neg': 0.0040194870446218145,
   'neu': 0.8361823639324092,
   'pos': 0.15979814902296907,
   'compound': 0.2838315484278386}),
 ('#arsd',
  {'neg': 0.06625747677822555,
   'neu': 0.8403805614939405,
   'pos': 0.0933619617278341,
   'compound': 0.06559285357805274}),
 ('#iowa',
  {'neg': 0.1051471591847877,
   'neu': 0.7490930511555067,
   'pos': 0.1457597896597056,
   'compound': 0.07929685423446334}),
 ('#biggboss',
  {'neg': 0.05310771801949638,
   'neu': 0.7550434295926761,
   'pos': 0.19184885238782745,
   'compound': 0.2500831740929352}),
 ('#thebachelor',
  {'neg': 0.09844065490137749,
   'neu': 0.792631254942909,
   'pos': 0.10892809015571361,
   'compound': 0.0170702485850578}),
 ('#asimriaz',
  {'neg': 0.06977859288060737,
   'neu': 0.7629025185663331,
   'pos': 0.16731888855305946,
   'compound': 0.15989261982882397}),
 ('#curecancer',
  {'neg': 0.2291470615972054,
   'neu': 0.5569524341721126,
   'pos': 0.213900504230682,
   'compound': -0.0994352485939529

In [62]:
print("time spent computing the sentiments, aggregating and normalizing: {:.4g}s".format(time1 + time2 + time3))

time spent computing the sentiments, aggregating and normalizing: 39.36s


#### Computing average sentiment on all tweets:

In [63]:
sentences_rdd  = words_f.select('filtered').rdd.map(lambda x: ' '.join(x[0]))

In [70]:
num_sentences = len(sentences)

In [64]:
t1 = time.time()
sentences = sentences_rdd.collect()
t2 = time.time()
time4 = t2-t1

In [71]:
t1 = time.time()
overall_sent = {'neg': 0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
for sentence in sentences:
    sentiment = sid.polarity_scores(sentence)    
    for key in overall_sent.keys():
        overall_sent[key] += sentiment[key]
        
    
for key in overall_sent.keys():
        overall_sent[key] /= num_sentences
overall_sent
t2 = time.time()
time5 = t2-t1

In [72]:
print("time spent computing the sentiments not using an rdd: {:.4g}s".format(time4 + time5))

time spent computing the sentiments not using an rdd: 82.31s


##### Conclusion:
We can clearly see that using an rdd and utilizing the map reduce scheme gives us faster performance! Also, the naive algorithm above doesn't even group by hashtags...